In [17]:
#imports
import warnings
import CoolProp.CoolProp as CP
from rocketcea.cea_obj_w_units import CEA_Obj
from scipy.optimize import fsolve
import cantera as ct
import numpy as np

warnings.filterwarnings("ignore")

In [4]:
p_exit = 10
P_c = 300

# Initialize CEA Rocket object
rocket = CEA_Obj(oxName='LOX', fuelName='RP-1', temperature_units='degK',
 cstar_units='m/sec', specific_heat_units='kJ/kg degK',
 sonic_velocity_units='m/s', enthalpy_units='J/kg',
 density_units='kg/m^3')


[_, Mf] = rocket.get_SpeciesMoleFractions(MR=2.6, Pc=P_c,frozen=1, frozenAtThroat=1)
temps = rocket.get_Temperatures(MR=2.6, frozen=1, frozenAtThroat=1, Pc=P_c)
velocities = rocket.get_SonicVelocities(MR=2.6, Pc=P_c, frozen=1, frozenAtThroat=1)
Mf = sorted(Mf.items(), key=lambda item: item[1], reverse=True)
Me = rocket.get_MachNumber(MR=2.6, Pc=P_c, frozen=1, frozenAtThroat=1)
for i in range(len(Mf)):
        print(f'{Mf[i][0]}: {Mf[i][1][1]*100:.3f}%')
[MW, gamma] = rocket.get_exit_MolWt_gamma(MR=2.6, frozen=1, frozenAtThroat=1, Pc=P_c)
print(f'Eqivalence Ratio: {rocket.get_eqratio(MR=2.6, Pc=P_c)[0]:.2f}')
print(f'Adiabatic Flame Temperature: {temps[0]:.2f} K')
print(f'Chamber Molecular Weight: {MW:.2f} g/mol')
print(f'Chamber Specific Heat Ratio: {gamma:.3f}')
print(f'Specific Impulse: {rocket.get_Isp(MR=2.6, Pc=P_c, frozen=1, frozenAtThroat=1):.2f} s')
print(f'Exit Temperature: {temps[2]:.2f} K')
print(f'Exit Mach Number: {Me:.3f}')
print(f'Exhaust Velocity: {velocities[2] * Me:.2f} m/s')



*CO: 31.831%
H2O: 30.879%
*CO2: 14.122%
*H2: 8.401%
*OH: 6.891%
*H: 3.797%
*O2: 2.383%
*O: 1.686%
HO2: 0.007%
HCO: 0.001%
COOH: 0.001%
H2O2: 0.001%
Eqivalence Ratio: 1.31
Adiabatic Flame Temperature: 3503.23 K
Chamber Molecular Weight: 23.45 g/mol
Chamber Specific Heat Ratio: 1.271
Specific Impulse: 331.58 s
Exit Temperature: 1104.59 K
Exit Mach Number: 4.431
Exhaust Velocity: 3126.41 m/s


In [5]:
gas = ct.Solution('c12H24_cmb_mech.yaml') # Cantera object for detailed combustion analysis
fuel = 'C12H24:1'  # RP-1
oxidizer = 'O2:1'  # Oxygen
MR = 2.6 # Mixture ratio O/F

gas.set_mixture_fraction(1/(1+MR), fuel, oxidizer) # Set mixture fraction. ct uses mixture fraction as fuel / (fuel + oxidizer)
phi = gas.equivalence_ratio(fuel, oxidizer, basis='mole') # Calculate equivalence ratio
print(f'Equivalence ratio: {phi:.2f}')

Equivalence ratio: 1.32


In [6]:
# Function to define the mole fraction equations
# X: array of mole fractions (X_C02, X_CO, X_H20)
# phi: equivalence ratio
def mole_fraction_equations(X):
    equation_1 =  X[0] + X[1] + X[2] - 1  # Sum of mole fractions equals 1
    equation_2 = X[2] - X[0] - X[1]  # Ratio of Carbon to Hydrogen
    equation_3 = 4/3 * phi * X[0] + 2/3 * phi * X[1] + 2/3 * phi * X[2] - 2 * X[2]  # Ratio of Hydrogen to Oxygen
    return [equation_1, equation_2, equation_3]

X0 = [1, 1, 1]  # Initial guess for mole fractions
X_solution = fsolve(mole_fraction_equations, X0)
print(f'Mole fractions: CO2 = {X_solution[0] * 100:.3f}%, CO = {X_solution[1]*100:.3f}%, H2O = {X_solution[2]*100:.3f}%')


Mole fractions: CO2 = 13.976%, CO = 36.024%, H2O = 50.000%


In [21]:
R = 8.314  # J/(mol·K)
T_ref = 298.15  # Reference temperature in K

H_f = [-393520, -110530, -241820]  # Standard enthalpies of formation at 298 K in J/mol for CO2, CO, H2O

def H_total(T):
    T_grid = np.linspace(T_ref, T, 101)
    dT = T_grid[1:] - T_grid[:-1]
    H = 0.0
    for i, species in enumerate(['CO2', 'CO', 'H2O']):
        cp_values = gas.species(species).thermo.cp(T)* R  # Get cp values in J/(mol·K)
        cp_interp = np.interp(T_grid, T, cp_values)
        H_species = np.sum(cp_interp[:-1] * dT)  # Integrate cp over temperature
        H += X_solution[i] * (H_species + H_f[i])  # Total enthalpy
    return H

H_react = -1598700  # J/mol, estimated enthalpy of C12H24

def energy_balance(T):
    return H_total(T) - H_react

T_initial_guess = 2500  # Initial guess for adiabatic flame temperature in K
T_ad_flame = fsolve(energy_balance, T_initial_guess)[0]
print(f'Adiabatic Flame Temperature (Cantera): {T_ad_flame:.2f} K')
   

ValueError: object of too small depth for desired array

In [7]:
T_O2 = 90.170  # Assumed temperature of lox in K
T_C12H24 = 298.15 # Assumed temperature of RP-1 in K
reactants = ct.ThermoPhase('c12H24_cmb_mech.yaml')
O2 = ct.Quantity(reactants, constant='HP')
O2.TPX = T_O2, P_c * 6894.76, 'O2:1'
O2.moles = 18
C12H24 = ct.Quantity(reactants, constant='HP') # Set Enthalpy Constant
C12H24.TPX = T_C12H24, P_c * 6894.76, 'C12H24:1'
C12H24.moles = phi 
products = O2 + C12H24
products.equilibrate('HP', solver='gibbs')
print(products.report())


  gas:

       temperature   3541.6 K
          pressure   2.0684e+06 Pa
           density   1.5928 kg/m^3
  mean mol. weight   22.676 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy       -4.1048e+05        -9.308e+06  J
   internal energy        -1.709e+06       -3.8755e+07  J
           entropy             11863        2.6902e+05  J/K
    Gibbs function       -4.2426e+07       -9.6206e+08  J
 heat capacity c_p            2037.7             46207  J/K
 heat capacity c_v              1671             37892  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
                H2         0.0079007          0.088868           -20.853
                 H         0.0019024          0.042796           -10.427
                 O          0.013507          0.019144           -15.811

In [8]:
sorted_mfs = sorted(products.mole_fraction_dict().items(), key=lambda x: x[1], reverse=True)
sorted_mfs = {sorted_mfs[i][0]: float(sorted_mfs[i][1]) for i in range(len(sorted_mfs)) if sorted_mfs[i][1] > 1e-3}
for sp, x in sorted_mfs.items():
    print(f" {sp}: {x*100:.3f}%")

cantera_to_coolprop = {
    "O2": "Oxygen",
    "CO2": "CarbonDioxide",
    "H2O": "Water",
    "CO": "CarbonMonoxide",
    "H2": "Hydrogen",
}

mapped_species = {}
for sp, x in sorted_mfs.items():
    if sp in cantera_to_coolprop:
        mapped_species[cantera_to_coolprop[sp]] = x

total = sum(mapped_species.values())
mapped_species = {k: v/total for k, v in mapped_species.items()}

cp_components = [f"{sp}[{x}]" for sp, x in mapped_species.items()]
mixture = "HEOS::" + "&".join(cp_components)

cp_mass = CP.PropsSI("Cpmass", "T", gas.T, "P", gas.P, mixture)
cv_mass = CP.PropsSI("Cvmass", "T", gas.T, "P", gas.P, mixture)

 CO: 31.886%
 H2O: 30.247%
 CO2: 13.018%
 H2: 8.887%
 OH: 7.255%
 H: 4.280%
 O2: 2.504%
 O: 1.914%


In [9]:
P_c = P_c * 6894.76 # Chamber Pressure in Pa
p_exit = p_exit * 6894.76 # Exit Pressure in Pa
specific_heat_ratio =  cp_mass / cv_mass
exit_mach = ((2/(specific_heat_ratio - 1)) * ((P_c/p_exit)**((specific_heat_ratio- 1)/specific_heat_ratio) - 1))**0.5
exit_T = products.T * (1 + (specific_heat_ratio - 1)/2 * exit_mach**2)**-1
exhuast_velocity = (2 * cp_mass *(products.T - exit_T))**0.5
I_sp = exhuast_velocity / 9.81
print(f'Adiabatic Flame Temperature: {products.T:.2f} K')
print(f'Chamber Molecular Weight: {products.mean_molecular_weight:.2f} g/mol')
print(f'Chamber Specific Heat Ratio: {specific_heat_ratio:.3f}')
print(f'Exit Mach Number: {exit_mach:.3f}')
print(f'Exit Temperature: {exit_T:.2f} K')
print(f'Exhaust Velocity: {exhuast_velocity:.2f} m/s')
print(f'Specific Impulse: {I_sp:.2f} s')



Adiabatic Flame Temperature: 3541.62 K
Chamber Molecular Weight: 22.68 g/mol
Chamber Specific Heat Ratio: 1.362
Exit Mach Number: 2.849
Exit Temperature: 1434.86 K
Exhaust Velocity: 2363.87 m/s
Specific Impulse: 240.96 s
